In [1]:
{1,3}#Definindo finais de semana
mes = "07"
sabado = "08/"+mes+"/2023"
domingo = "09/"+mes+"/2023"

In [2]:
#PACOTES
import pandas as pd #manipulação de dados
import numpy as np #matematica

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import unidecode #arrumar strings

import re #regex

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
#lendo arquivo txt e transformando em dataframe cada mensagem
def read_file(file):
    '''Reads Whatsapp text file into a list of strings'''
    x = open(file, 'r', encoding = 'utf-8')
    y = x.read()
    content = y.splitlines()
    return content

chat = read_file('_chat.txt')
df = pd.DataFrame({'chat':chat})
df

,chat
0,[01/07/2023 04:31:07] Fernando Ribeiro Sá: 01/...
1,[01/07/2023 04:55:35] Ana Gabriela Gewehr Vale...
2,[01/07/2023 05:17:10] Rodrigo Miyashiro Nunes ...
3,[01/07/2023 05:21:55] Yuri Anderson Pacheco de...
4,[01/07/2023 05:22:51] André da Costa Aguiar Ve...
...,...
865,[10/07/2023 06:20:41] Bruna Daibert Danesin: 1...
866,[10/07/2023 06:25:02] Karen Manoela Rosa: 10/0...
867,[10/07/2023 06:35:02] Bruce Ye Man Chow: 10/07...
868,[10/07/2023 06:36:31] Rogério Sidney Thon: 10/...


# CAPTURANDO INFORMAÇÕES:

In [5]:
#DIA QUE FOI ENVIADA A MENSAGEM
dia = []
for i in range(0, len(df['chat'])):
    dia.append(df['chat'][i][1:11]) #pegando exatamente aonde parte da data esta
df['dia'] = dia
df['dia'].unique()

array(['01/07/2023', '02/07/2023', '03/07/2023', '04/07/2023',
       '05/07/2023', '06/07/2023', '07/07/2023', '08/07/2023',
       '09/07/2023', '10/07/2023'], dtype=object)

In [6]:
#NOME DO ANESTESISTA
nome = []
count = 0
for i in range(0, len(df['chat'])):
    loc = df['chat'][i].find(']')+1 #digito data
    loc2 = df['chat'][i][loc+1:].find(':')+1 #digito nome
    nome.append(unidecode.unidecode(df['chat'][i][loc+1:loc+loc2].upper()))
    count += 1
    #print(count, "->", unidecode.unidecode(chat[i][loc+1:loc+loc2].upper()))
df['nome'] = nome
df['nome'].unique()

array(['FERNANDO RIBEIRO SA', 'ANA GABRIELA GEWEHR VALE SALMEN',
       'RODRIGO MIYASHIRO NUNES DOS SANTOS',
       'YURI ANDERSON PACHECO DE LIMA', 'ANDRE DA COSTA AGUIAR VENTURA',
       'ORLANDO DOMINGUES DE ARAUJO PONTES',
       'GABRIEL ELIAS GEWEHR SALMEN RAFFOUL', 'GABRIELA SANTOS MENEZES',
       'RODRIGO ANTONIO REY GONZALEZ', 'THIAGO BARRETO MARQUES DA SILVA',
       'MARIA CECILIA DOMINGUES DA SILVA SOARES',
       'FRANCISCO ANTONIO ALVES BONATTO',
       'VIVIAN CRISTINE DOS ANJOS E SILVA', 'LEONARDO DE PAULA SOUZA',
       'DEBORA ALVES AUDE', 'RODRIGO GOMES MINAS NOVAS',
       'EDUARDO ARBACHE BEZERRA', 'DANIEL  MASUZAKI WONG',
       'VITOR OLIVEIRA ANDRE', 'HELENA ARAUJO DAMASCENO',
       'GUILHERME SILVEIRA LOPES', 'ROMULO SPOLIDORIO',
       'PEDRO HENRIQUE DINIZ BOTELHO', 'RODRIGO ANDRADE CARMELLO',
       'CAMILA CHIARANTANO HARRISON', 'ALEXANDRE AMARAL LUCAS',
       'FABIANA BARBOSA BASTOS', 'DENIO LOPES DE ALMEIDA',
       'CLAUDIO MUELLER KAKUDA', 'ERICK OL

In [7]:
#MENSAGEM ESCRITA
mensagem = []
for i in range(0, len(df)):
    digito_data = df['chat'][i].find(']') #aonde acaba a data
    digito_nome = df['chat'][i][digito_data+1:].find(':')
    mensagem_escrevida = df['chat'][i][digito_data+digito_nome+3:]
    mensagem.append(mensagem_escrevida)
    
df['mensagem'] = mensagem
df['mensagem'].unique()

array(['01/07 VNS Ribeiro entrada 04:20',
       '01/07 Morumbi - Vale entrada 4:50',
       '01/07 ITAIM - Miyashiro entrada 05:15',
       '01/07 Morumbi - Yuri entrada 05:20',
       '01/07 VNS Ventura entrada 5:20',
       '01/07 ANÁLIA  - Orlando entrada 05:30',
       '01/07 Analia Salmen entrada 05:30',
       '01/07 Anália - G.Menezes entrada 05:30 (deslocada Jabaquara)',
       '01/07 SANTA - Rey entrada 05:30',
       '01/07 Villa - Naldo - entrada 05:35',
       '01/07 VILLA - M. Cecília entrada 05:35',
       '01/07 ANÁLIA - Francisco entrada 05:35',
       '01/07 VILLA Vivian entrada 05:40',
       '01/07 Jabaquara Leonardo entrada  05:45',
       '01/07 Morumbi Débora entrada 05:40',
       '01/07 SANTA - Gomes entrada 05:20',
       '01/07 Itaim - Bezerra entrada as 06:00',
       '01/07 Morumbi - Wong entrada 06:00',
       '01/07 Jabaquara Vitor entrada 06:00',
       '01/07 Anália- Helena- entrada 06:00',
       '01/07 Itaim - Silveira entrada 06:00',
       '01/07 Vi

In [8]:
#DATA DO PLANTÃO
df['controle'] = ''
df['data'] = ''
invalido =0 
for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar registros com uma data apenas
        df['data'][i] = lista_datas[0]
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'PROB'
        invalido += 1

print('mais de uma data: '+str(invalido))

df['data'].unique()

mais de uma data: 4


array(['01/07', '', '01/7', '02/07', '29/06', '03/07', '03/06', '3/7',
       '03/7', '04/07', '04/7', '05/07', '05/08', '06/07', '05/7',
       '04/06', '5/7', '6/7', '06/7', '07/07', '07/7', '7/7', '06/08',
       '08/07', '08/7', '8/7', '09/07', '10/07'], dtype=object)

# Obs registros:

In [9]:
df[df['controle'] == 'PROB']

,chat,dia,nome,mensagem,controle,data
41,[01/07/2023 11:15:39] Alexandre Amaral Lucas: ...,01/07/2023,ALEXANDRE AMARAL LUCAS,01:07 Aviccena - alexandre Amaral saída 11:10,PROB,
197,[03/07/2023 15:18:56] Gabriel Elias Gewehr Sal...,03/07/2023,GABRIEL ELIAS GEWEHR SALMEN RAFFOUL,Osasco Salmen (pós plantão Osasco) saída 14:35,PROB,
361,[04/07/2023 22:17:51] Rodrigo Antonio Rey Gonz...,04/07/2023,RODRIGO ANTONIO REY GONZALEZ,OSASCO - Rey saida 22:00,PROB,
761,[08/07/2023 10:40:47] Sheila Itiroco Monte: (D...,08/07/2023,SHEILA ITIROCO MONTE,(Deslocada Itaim),PROB,


In [10]:
#CAPTURANDO UNIDADE
df['unidade'] = ''
df['Q_unidades'] = ''
unidades = []

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    lista_unidades = re.findall(r'HSI|ANALIA|AVICCENA|ITAIM|JABA|MATERNIDADE|MORUMBI|OSASCO|VILLA|VNS|SANTA', mensagem)
    quantidade_unidades = len(lista_unidades)
    unidades.append(lista_unidades)
    
    df['Q_unidades'][i] = quantidade_unidades
    
    if quantidade_unidades >= 1: #capturando com unidades
        df['unidade'][i] = lista_unidades[0]
    elif quantidade_unidades == 0:
        df['controle'][i] = 'PROB'
        
print(df['Q_unidades'].value_counts(),'\n')
print("prob: "+str(len(df[df['controle'] == 'PROB'])))

1    811
2     57
0      2
Name: Q_unidades, dtype: int64 

prob: 6


In [11]:
prob = df[df['controle'] == 'PROB']
prob

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades
41,[01/07/2023 11:15:39] Alexandre Amaral Lucas: ...,01/07/2023,ALEXANDRE AMARAL LUCAS,01:07 Aviccena - alexandre Amaral saída 11:10,PROB,,AVICCENA,1
197,[03/07/2023 15:18:56] Gabriel Elias Gewehr Sal...,03/07/2023,GABRIEL ELIAS GEWEHR SALMEN RAFFOUL,Osasco Salmen (pós plantão Osasco) saída 14:35,PROB,,OSASCO,2
224,[03/07/2023 20:13:30] Andre Mortari Plá Gil: 0...,03/07/2023,ANDRE MORTARI PLA GIL,03/07 EINSTEIN André saída 19:40h,PROB,03/07,,0
312,[04/07/2023 15:38:28] Thiago Francisco Martins...,04/07/2023,THIAGO FRANCISCO MARTINS,04/07 ANALI Thiago Martins saida 15:25,PROB,04/07,,0
361,[04/07/2023 22:17:51] Rodrigo Antonio Rey Gonz...,04/07/2023,RODRIGO ANTONIO REY GONZALEZ,OSASCO - Rey saida 22:00,PROB,,OSASCO,1
761,[08/07/2023 10:40:47] Sheila Itiroco Monte: (D...,08/07/2023,SHEILA ITIROCO MONTE,(Deslocada Itaim),PROB,,ITAIM,1


In [12]:
#RETIRANDO REGISTROS DE DESLOCADOS, DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE

#preparando para reconhecer corretamente
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('(',' ( '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace(')',' ) '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.upper())
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCADA','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCAMENTO','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSADA','DISPENSADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSA ','DISPENSADO'))
df['sit'] = ''


for i in range(0, len(df)):
    mensagem = df['mensagem'][i]
    observacoes = re.findall(r'DESLOCADO|DISPENSADO|SAUDE|DISTANCIA', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes > 0:
        df['controle'][i] = 'PROB'
        
for i in range(0, len(df)):
    mensagem = df['mensagem'][i]
    deslocado = re.findall(r'DESLOCADO', mensagem)
    dispensado = re.findall(r'DISPENSADO', mensagem)
    distancia = re.findall(r'DISTANCIA', mensagem)
    
    if len(deslocado) > 0:
        df['sit'][i] = "DESLOCADO"
    elif len(dispensado) > 0:
        df['sit'][i] = "DISPENSADO"
    elif len(distancia)  > 0:
        df['sit'][i] = "DISTANCIA"
    else:
        df['sit'][i] = ''
        
print(df['controle'].value_counts()) 
prob = df[df['controle'] == 'PROB']


 

        805
PROB     65
Name: controle, dtype: int64


In [13]:
prob

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit
7,[01/07/2023 05:30:43] Gabriela Santos Menezes:...,01/07/2023,GABRIELA SANTOS MENEZES,01/07 ANÁLIA - G.MENEZES ENTRADA 05:30 ( DESL...,PROB,01/07,ANALIA,2,DESLOCADO
27,[01/07/2023 06:55:12] Denio Lopes de Almeida: ...,01/07/2023,DENIO LOPES DE ALMEIDA,01/07 OSASCO - DÊNIO ENTRADA 06:00 ( DESLOCAD...,PROB,01/07,OSASCO,2,DESLOCADO
38,[01/07/2023 10:27:51] Caroline Firmino Lemes d...,01/07/2023,CAROLINE FIRMINO LEMES DA SILVA,01/07 OSASCO - LEMES SAIDA 10:25 ( DESLOCADO ...,PROB,01/07,OSASCO,2,DESLOCADO
41,[01/07/2023 11:15:39] Alexandre Amaral Lucas: ...,01/07/2023,ALEXANDRE AMARAL LUCAS,01:07 AVICCENA - ALEXANDRE AMARAL SAÍDA 11:10,PROB,,AVICCENA,1,
54,[01/07/2023 13:13:59] Denio Lopes de Almeida: ...,01/07/2023,DENIO LOPES DE ALMEIDA,01/07 OSASCO - DÊNIO SAÍDA 13:10 ( DESLOCADO ...,PROB,01/07,OSASCO,2,DESLOCADO
55,[01/07/2023 13:23:26] Ronald Figueiredo Gomes:...,01/07/2023,RONALD FIGUEIREDO GOMES,01/07 JABAQUARA - RONALD SAÍDA 13:15 ( PÓS PL...,PROB,01/07,JABA,2,DESLOCADO
69,[01/07/2023 15:00:58] Gabriela Santos Menezes:...,01/07/2023,GABRIELA SANTOS MENEZES,01/07 ANÁLIA - G.MENEZES SAÍDA 15:00 ( DESLOC...,PROB,01/07,ANALIA,2,DESLOCADO
93,[01/07/2023 17:40:55] André da Costa Aguiar Ve...,01/07/2023,ANDRE DA COSTA AGUIAR VENTURA,01/07 ANALIA VENTURA SAIDA 17:40 ( DESLOCAD...,PROB,01/07,ANALIA,2,DESLOCADO
118,[01/07/2023 20:58:55] Bianca Priscila Buniotti...,01/07/2023,BIANCA PRISCILA BUNIOTTI,01/07 VILLA - BIANCA SAIDA 16:45 ( DESLOCADO ...,PROB,01/07,VILLA,2,DESLOCADO
144,[03/07/2023 04:35:22] Kaiê Fernandes: 03/07 Os...,03/07/2023,KAIE FERNANDES,03/07 OSASCO KAIE ENTRADA 4:20 ( DESLOCADO IT...,PROB,03/07,OSASCO,2,DESLOCADO


In [14]:
#VERIFICANDO SE É EXTRA
df["extra"] = ''

for i in range(0, len(df)):
    if 'EXTRA' in df['mensagem'][i].upper(): #se contiver a palavra "extra"
        df['extra'][i] = 'extra' #marque na coluna extra como extra
        
print(df['extra'].value_counts()        )

         848
extra     22
Name: extra, dtype: int64


# Tipo entrada e saida:

In [15]:
#Indentificando se é um registro de entrada ou saida
df['tipo'] = ''
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('SAÍDA','SAIDA'))
invalido = 0
for i in range(0, len(df)):
    mensagem = df['mensagem'][i]
    entradas_saidas = re.findall(r'ENTRADA|SAIDA', mensagem)
    quantidade_entradas_saidas = len(entradas_saidas)
    
    if quantidade_entradas_saidas != 1: #capturando apenas com uma informação escrita
        df['controle'][i] = 'PROB'
        invalido += 1
    else: #restante com mais de uma informação
        df['tipo'][i] = entradas_saidas[0]
        
print(df['tipo'].value_counts())        
print(df['controle'].value_counts())
print('entrada e saída junto: '+str(invalido))

prob = df[df['controle'] == 'PROB']


SAIDA      523
ENTRADA    332
            15
Name: tipo, dtype: int64
        798
PROB     72
Name: controle, dtype: int64
entrada e saída junto: 15


In [16]:
prob

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo
7,[01/07/2023 05:30:43] Gabriela Santos Menezes:...,01/07/2023,GABRIELA SANTOS MENEZES,01/07 ANÁLIA - G.MENEZES ENTRADA 05:30 ( DESL...,PROB,01/07,ANALIA,2,DESLOCADO,,ENTRADA
27,[01/07/2023 06:55:12] Denio Lopes de Almeida: ...,01/07/2023,DENIO LOPES DE ALMEIDA,01/07 OSASCO - DÊNIO ENTRADA 06:00 ( DESLOCAD...,PROB,01/07,OSASCO,2,DESLOCADO,,ENTRADA
38,[01/07/2023 10:27:51] Caroline Firmino Lemes d...,01/07/2023,CAROLINE FIRMINO LEMES DA SILVA,01/07 OSASCO - LEMES SAIDA 10:25 ( DESLOCADO ...,PROB,01/07,OSASCO,2,DESLOCADO,,SAIDA
41,[01/07/2023 11:15:39] Alexandre Amaral Lucas: ...,01/07/2023,ALEXANDRE AMARAL LUCAS,01:07 AVICCENA - ALEXANDRE AMARAL SAIDA 11:10,PROB,,AVICCENA,1,,,SAIDA
45,[01/07/2023 11:22:04] Ricardo Barbosa Viana Fi...,01/07/2023,RICARDO BARBOSA VIANA FILHO,01/07 SANTA ISABEL - BARBOSA 11:15 ASSUMO ROTI...,PROB,01/07,SANTA,1,,,
54,[01/07/2023 13:13:59] Denio Lopes de Almeida: ...,01/07/2023,DENIO LOPES DE ALMEIDA,01/07 OSASCO - DÊNIO SAIDA 13:10 ( DESLOCADO ...,PROB,01/07,OSASCO,2,DESLOCADO,,SAIDA
55,[01/07/2023 13:23:26] Ronald Figueiredo Gomes:...,01/07/2023,RONALD FIGUEIREDO GOMES,01/07 JABAQUARA - RONALD SAIDA 13:15 ( PÓS PL...,PROB,01/07,JABA,2,DESLOCADO,,SAIDA
69,[01/07/2023 15:00:58] Gabriela Santos Menezes:...,01/07/2023,GABRIELA SANTOS MENEZES,01/07 ANÁLIA - G.MENEZES SAIDA 15:00 ( DESLOC...,PROB,01/07,ANALIA,2,DESLOCADO,,SAIDA
89,[01/07/2023 17:16:37] Nara Yamane dos Santos: ...,01/07/2023,NARA YAMANE DOS SANTOS,01/07 JABAQUARA - NARA 17:10H,PROB,01/07,JABA,1,,,
93,[01/07/2023 17:40:55] André da Costa Aguiar Ve...,01/07/2023,ANDRE DA COSTA AGUIAR VENTURA,01/07 ANALIA VENTURA SAIDA 17:40 ( DESLOCAD...,PROB,01/07,ANALIA,2,DESLOCADO,,SAIDA


# Data:

In [17]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = ''

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio

In [18]:
df

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora
0,[01/07/2023 04:31:07] Fernando Ribeiro Sá: 01/...,01/07/2023,FERNANDO RIBEIRO SA,01/07 VNS RIBEIRO ENTRADA 04:20,,01/07,VNS,1,,,ENTRADA,VNS RIBEIRO ENTRADA 04:20
1,[01/07/2023 04:55:35] Ana Gabriela Gewehr Vale...,01/07/2023,ANA GABRIELA GEWEHR VALE SALMEN,01/07 MORUMBI - VALE ENTRADA 4:50,,01/07,MORUMBI,1,,,ENTRADA,MORUMBI - VALE ENTRADA 4:50
2,[01/07/2023 05:17:10] Rodrigo Miyashiro Nunes ...,01/07/2023,RODRIGO MIYASHIRO NUNES DOS SANTOS,01/07 ITAIM - MIYASHIRO ENTRADA 05:15,,01/07,ITAIM,1,,,ENTRADA,ITAIM - MIYASHIRO ENTRADA 05:15
3,[01/07/2023 05:21:55] Yuri Anderson Pacheco de...,01/07/2023,YURI ANDERSON PACHECO DE LIMA,01/07 MORUMBI - YURI ENTRADA 05:20,,01/07,MORUMBI,1,,,ENTRADA,MORUMBI - YURI ENTRADA 05:20
4,[01/07/2023 05:22:51] André da Costa Aguiar Ve...,01/07/2023,ANDRE DA COSTA AGUIAR VENTURA,01/07 VNS VENTURA ENTRADA 5:20,,01/07,VNS,1,,,ENTRADA,VNS VENTURA ENTRADA 5:20
...,...,...,...,...,...,...,...,...,...,...,...,...
865,[10/07/2023 06:20:41] Bruna Daibert Danesin: 1...,10/07/2023,BRUNA DAIBERT DANESIN,10/07 SANTA - BRUNA ENTRADA 06:00,,10/07,SANTA,1,,,ENTRADA,SANTA - BRUNA ENTRADA 06:00
866,[10/07/2023 06:25:02] Karen Manoela Rosa: 10/0...,10/07/2023,KAREN MANOELA ROSA,10/07 OSASCO KROSA ENTRADA 06:00,,10/07,OSASCO,1,,,ENTRADA,OSASCO KROSA ENTRADA 06:00
867,[10/07/2023 06:35:02] Bruce Ye Man Chow: 10/07...,10/07/2023,BRUCE YE MAN CHOW,10/07 MORUMBI - BRUCE ENTRADA 06:00,,10/07,MORUMBI,1,,,ENTRADA,MORUMBI - BRUCE ENTRADA 06:00
868,[10/07/2023 06:36:31] Rogério Sidney Thon: 10/...,10/07/2023,ROGERIO SIDNEY THON,10/07 MORUMBI ROGÉRIO ENTRADA 5:20H,,10/07,MORUMBI,1,,,ENTRADA,MORUMBI ROGÉRIO ENTRADA 5:20H


In [19]:
#arrumando data d/mm (6/10, não 06/10)
for i in range(0, len(df)):
    if df['data'][i] != "" and df['data'][i][1] == '/':
        df['data'][i] = '0'+df['data'][i]
df['data'].unique()

array(['01/07', '', '01/7', '02/07', '29/06', '03/07', '03/06', '03/7',
       '04/07', '04/7', '05/07', '05/08', '06/07', '05/7', '04/06',
       '06/7', '07/07', '07/7', '06/08', '08/07', '08/7', '09/07',
       '10/07'], dtype=object)

In [20]:
#somando informações restantes na data: dd/MM/yyyy
for i in range(0, len(df)):
    df['data'][i] = df['data'][i][0:2] + "/"+ mes +"/2023" 
    
print(df['data'].unique())

['01/07/2023' '/07/2023' '02/07/2023' '29/07/2023' '03/07/2023'
 '04/07/2023' '05/07/2023' '06/07/2023' '07/07/2023' '08/07/2023'
 '09/07/2023' '10/07/2023']


In [21]:
df['sem_ou_fds'] = 'semana'

df.loc[(df['data'] == sabado) | (df['data'] == domingo), 'sem_ou_fds'] = 'fds'
df.loc[(df['data'] == "01/07/2023") | (df['data'] == "02/07/2023"), 'sem_ou_fds'] = 'fds'

df['sem_ou_fds'].value_counts()

semana    601
fds       269
Name: sem_ou_fds, dtype: int64

In [22]:
df

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds
0,[01/07/2023 04:31:07] Fernando Ribeiro Sá: 01/...,01/07/2023,FERNANDO RIBEIRO SA,01/07 VNS RIBEIRO ENTRADA 04:20,,01/07/2023,VNS,1,,,ENTRADA,VNS RIBEIRO ENTRADA 04:20,fds
1,[01/07/2023 04:55:35] Ana Gabriela Gewehr Vale...,01/07/2023,ANA GABRIELA GEWEHR VALE SALMEN,01/07 MORUMBI - VALE ENTRADA 4:50,,01/07/2023,MORUMBI,1,,,ENTRADA,MORUMBI - VALE ENTRADA 4:50,fds
2,[01/07/2023 05:17:10] Rodrigo Miyashiro Nunes ...,01/07/2023,RODRIGO MIYASHIRO NUNES DOS SANTOS,01/07 ITAIM - MIYASHIRO ENTRADA 05:15,,01/07/2023,ITAIM,1,,,ENTRADA,ITAIM - MIYASHIRO ENTRADA 05:15,fds
3,[01/07/2023 05:21:55] Yuri Anderson Pacheco de...,01/07/2023,YURI ANDERSON PACHECO DE LIMA,01/07 MORUMBI - YURI ENTRADA 05:20,,01/07/2023,MORUMBI,1,,,ENTRADA,MORUMBI - YURI ENTRADA 05:20,fds
4,[01/07/2023 05:22:51] André da Costa Aguiar Ve...,01/07/2023,ANDRE DA COSTA AGUIAR VENTURA,01/07 VNS VENTURA ENTRADA 5:20,,01/07/2023,VNS,1,,,ENTRADA,VNS VENTURA ENTRADA 5:20,fds
...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,[10/07/2023 06:20:41] Bruna Daibert Danesin: 1...,10/07/2023,BRUNA DAIBERT DANESIN,10/07 SANTA - BRUNA ENTRADA 06:00,,10/07/2023,SANTA,1,,,ENTRADA,SANTA - BRUNA ENTRADA 06:00,semana
866,[10/07/2023 06:25:02] Karen Manoela Rosa: 10/0...,10/07/2023,KAREN MANOELA ROSA,10/07 OSASCO KROSA ENTRADA 06:00,,10/07/2023,OSASCO,1,,,ENTRADA,OSASCO KROSA ENTRADA 06:00,semana
867,[10/07/2023 06:35:02] Bruce Ye Man Chow: 10/07...,10/07/2023,BRUCE YE MAN CHOW,10/07 MORUMBI - BRUCE ENTRADA 06:00,,10/07/2023,MORUMBI,1,,,ENTRADA,MORUMBI - BRUCE ENTRADA 06:00,semana
868,[10/07/2023 06:36:31] Rogério Sidney Thon: 10/...,10/07/2023,ROGERIO SIDNEY THON,10/07 MORUMBI ROGÉRIO ENTRADA 5:20H,,10/07/2023,MORUMBI,1,,,ENTRADA,MORUMBI ROGÉRIO ENTRADA 5:20H,semana


# Hora:

In [23]:
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

In [24]:
invalido = 0
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)

    if quantidade_horas == 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'PROB'
        invalido += 1
        
print(df['controle'].value_counts())
print('mais de uma hora ou nenhuma hora: '+str(invalido))
prob = df[df['controle'] == "PROB"]

 

        794
PROB     76
Name: controle, dtype: int64
mais de uma hora ou nenhuma hora: 17


In [25]:
len(prob)

76

In [26]:
len(df[df['hora'] == ""])

17

In [27]:
#adicionando 0 no primeiro algorismo dos valores h:mm
for i in range(0, len(df)):
    if df['hora'][i] != "" and df['hora'][i][1] == ":":
        df['hora'][i] = "0"+df['hora'][i]
        
df['hora'].unique()

array(['04:20', '04:50', '05:15', '05:20', '05:30', '05:35', '05:40',
       '05:45', '06:00', '04:40', '06:20', '06:30', '06:50', '09:48',
       '10:00', '10:20', '10:25', '10:40', '11:00', '', '11:15', '11:40',
       '12:15', '12:38', '12:50', '13:10', '13:15', '13:20', '13:35',
       '13:45', '13:40', '14:15', '13:30', '14:30', '14:40', '14:42',
       '14:45', '15:00', '15:10', '14:50', '15:30', '15:35', '15:45',
       '15:50', '15:55', '16:20', '16:40', '16:45', '16:50', '17:07',
       '17:00', '17:10', '17:20', '16:55', '17:40', '17:50', '18:00',
       '18:10', '18:25', '18:15', '18:35', '18:20', '17:45', '18:50',
       '18:48', '18:58', '19:15', '19:45', '20:00', '20:10', '16:15',
       '06:15', '08:00', '07:30', '07:00', '11:50', '12:30', '12:40',
       '21:00', '06:45', '17:15', '22:20', '04:45', '05:00', '07:50',
       '15:40', '12:20', '06:10', '12:45', '13:47', '14:16', '14:25',
       '14:35', '14:20', '16:10', '16:35', '19:10', '19:35', '19:30',
       '19:40', 

# Hora final:

In [28]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [29]:
len(df)

870

In [30]:
prob = df[df['controle'] == "PROB"]
prob.reset_index(drop=True, inplace=True)
prob_hora = df[df['hora'] == ""] #zero ou mais de uma hora
prob_hora.reset_index(drop=True, inplace=True)
df = df[df['hora'] != ""] #df passa a ter ainda os prob, mas sem incluir os prob de hora
df.reset_index(drop=True, inplace=True)
print("prob: "+str(len(prob)))
print("prob_hora: "+str(len(prob_hora)))
print("df: "+str(len(df)))

prob: 76
prob_hora: 17
df: 853


In [31]:
#criando uma coluna datetime para fazer calculos de horas e minutos
df['hora2'] = df['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

#verificando se esta tudo certo
print(df['hora2'].dt.year.unique())
print(df['hora2'].dt.month.unique())
print(df['hora2'].dt.day.unique())
print(df['hora2'].dt.hour.unique())

for i in range(0, len(df)):
    minuto = int(df['hora'][i][3:])
    
    if minuto != 0 and minuto != 30:
        if minuto < 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (0) subtraindo quantos minutos tem entre o momento e HH:00
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-minuto))[11:-3]
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (30) acrecentando quantos minutos faltam para HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(30-minuto)))[11:-3]
            else:
                df['controle'][i] = 'PROB'
                
        elif minuto > 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (30) subtraindo quando minutos tem entre o momento e HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-(minuto-30)))[11:-3]   
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (60) acrecentando quantos minutos faltam para HH:60
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(60-minuto)))[11:-3]            
            else:
                df['controle'][i] = 'PROB'
        else:
            df['controle'][i] = 'PROB'
            
    elif minuto == 0 or minuto == 30:
        df['hora'][i] = str(df['hora2'][i])[11:-3] 
        
    else:
        df['controle'][i] = 'PROB'
        
df['hora'].unique()

[1900]
[1]
[1]
[ 4  5  6  9 10 11 12 13 14 15 16 17 18 19 20  8  7 21 22 23  0  1]


array(['04:00', '04:30', '05:00', '05:30', '06:00', '06:30', '10:00',
       '10:30', '11:00', '11:30', '11:15', '12:00', '12:30', '13:00',
       '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30',
       '17:00', '17:30', '17:10', '18:00', '18:30', '19:00', '19:30',
       '20:00', '20:30', '08:00', '07:30', '07:00', '21:00', '22:30',
       '05:40', '21:30', '22:00', '23:00', '23:30', '08:30', '00:00',
       '01:00', '01:30', '02:00', '09:30'], dtype=object)

# Setor:

In [32]:
#reconhendo em qual periodo do dia aconteceu o plantão
df['ENTRADA'] = ''
df['SAIDA'] = ''
df['PERIODO'] = ''

for i in range(0, len(df)):
    minuto = int(df['hora'][i][:2])
    
    if df['tipo'][i] == 'ENTRADA':
        df['ENTRADA'][i] = df['hora'][i]
        if minuto < 7: #periodo 6h
            df['PERIODO'][i] = '6h'
            df['SAIDA'][i] = '07:00'
        elif minuto >= 7 and minuto < 13: #periodo manha
            df['PERIODO'][i] = 'manha'
            df['SAIDA'][i] = '13:00'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['PERIODO'][i] = 'tarde'
            df['SAIDA'][i] = '19:00'
        else:
            df['controle'][i] = 'PROB'
    elif df['tipo'][i] == 'SAIDA':
        df['SAIDA'][i] = df['hora'][i]
        if minuto >= 7 and minuto < 13: #periodo manha
            df['ENTRADA'][i] = '07:00'
            df['PERIODO'][i] = 'manha'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['ENTRADA'][i] = '13:00'
            df['PERIODO'][i] = 'tarde'
        elif minuto >= 19: #periodo noturno
            df['ENTRADA'][i] = '19:00'
            df['PERIODO'][i] = 'noturno saida'
        elif minuto <= 2: #periodo noturno
            df['ENTRADA'][i] = '19:00'
            df['PERIODO'][i] = 'noturno saida'
        else:
            df['controle'][i] = 'PROB'
    else:
        df['controle'][i] = 'PROB'
        
df['PERIODO'].value_counts()

print(df['controle'].value_counts())

prob = df[df['controle'] == "PROB"]
prob.reset_index(drop=True, inplace=True)
print("probs:", len(prob))


        794
PROB     59
Name: controle, dtype: int64
probs: 59


In [33]:
prob

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO
0,[01/07/2023 05:30:43] Gabriela Santos Menezes:...,01/07/2023,GABRIELA SANTOS MENEZES,01/07 ANÁLIA - G.MENEZES ENTRADA 05:30 ( DESL...,PROB,01/07/2023,ANALIA,2,DESLOCADO,,ENTRADA,05:30,fds,1900-01-01 05:30:00,05:30,07:00,6h
1,[01/07/2023 06:55:12] Denio Lopes de Almeida: ...,01/07/2023,DENIO LOPES DE ALMEIDA,01/07 OSASCO - DÊNIO ENTRADA 06:00 ( DESLOCAD...,PROB,01/07/2023,OSASCO,2,DESLOCADO,,ENTRADA,06:00,fds,1900-01-01 06:00:00,06:00,07:00,6h
2,[01/07/2023 10:27:51] Caroline Firmino Lemes d...,01/07/2023,CAROLINE FIRMINO LEMES DA SILVA,01/07 OSASCO - LEMES SAIDA 10:25 ( DESLOCADO ...,PROB,01/07/2023,OSASCO,2,DESLOCADO,,SAIDA,10:30,fds,1900-01-01 10:25:00,07:00,10:30,manha
3,[01/07/2023 11:22:04] Ricardo Barbosa Viana Fi...,01/07/2023,RICARDO BARBOSA VIANA FILHO,01/07 SANTA ISABEL - BARBOSA 11:15 ASSUMO ROTI...,PROB,01/07/2023,SANTA,1,,,,11:15,fds,1900-01-01 11:15:00,,,
4,[01/07/2023 13:13:59] Denio Lopes de Almeida: ...,01/07/2023,DENIO LOPES DE ALMEIDA,01/07 OSASCO - DÊNIO SAIDA 13:10 ( DESLOCADO ...,PROB,01/07/2023,OSASCO,2,DESLOCADO,,SAIDA,13:30,fds,1900-01-01 13:10:00,13:00,13:30,tarde
5,[01/07/2023 13:23:26] Ronald Figueiredo Gomes:...,01/07/2023,RONALD FIGUEIREDO GOMES,01/07 JABAQUARA - RONALD SAIDA 13:15 ( PÓS PL...,PROB,01/07/2023,JABA,2,DESLOCADO,,SAIDA,13:30,fds,1900-01-01 13:15:00,13:00,13:30,tarde
6,[01/07/2023 15:00:58] Gabriela Santos Menezes:...,01/07/2023,GABRIELA SANTOS MENEZES,01/07 ANÁLIA - G.MENEZES SAIDA 15:00 ( DESLOC...,PROB,01/07/2023,ANALIA,2,DESLOCADO,,SAIDA,15:00,fds,1900-01-01 15:00:00,13:00,15:00,tarde
7,[01/07/2023 17:16:37] Nara Yamane dos Santos: ...,01/07/2023,NARA YAMANE DOS SANTOS,01/07 JABAQUARA - NARA 17:10H,PROB,01/07/2023,JABA,1,,,,17:10,fds,1900-01-01 17:10:00,,,
8,[01/07/2023 17:40:55] André da Costa Aguiar Ve...,01/07/2023,ANDRE DA COSTA AGUIAR VENTURA,01/07 ANALIA VENTURA SAIDA 17:40 ( DESLOCAD...,PROB,01/07/2023,ANALIA,2,DESLOCADO,,SAIDA,18:00,fds,1900-01-01 17:40:00,13:00,18:00,tarde
9,[01/07/2023 20:58:55] Bianca Priscila Buniotti...,01/07/2023,BIANCA PRISCILA BUNIOTTI,01/07 VILLA - BIANCA SAIDA 16:45 ( DESLOCADO ...,PROB,01/07/2023,VILLA,2,DESLOCADO,,SAIDA,17:00,fds,1900-01-01 16:45:00,13:00,17:00,tarde


In [34]:
prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
print("prob: "+str(len(prob)))
print("prob_hora: "+str(len(prob_hora)))
print("df: "+str(len(df)))

prob: 59
prob_hora: 17
df: 853


In [35]:
prob_dura = df[(df['SAIDA'] == "" )|  (df['ENTRADA'] == "")] #prob_dura: sem entrada ou saída
df = df[df['ENTRADA'] != ""]
df = df[df['SAIDA'] != ""]  ## retiro sem ENTRADA OU SEM SAÍDA
df.reset_index(drop=True, inplace=True)
prob_dura.reset_index(drop=True, inplace=True)

In [36]:
print(len(df))
print(len(prob_dura))


850
3


In [37]:
problemao = pd.concat([prob_hora, prob_dura]) #problemao = prob mais de uma hora, nenhuma hora , nenhuma entrada ou saída
problemao.reset_index(drop=True, inplace=True) #todo problemao já não faz parte de df
problemao

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO
0,[01/07/2023 11:15:39] Alexandre Amaral Lucas: ...,01/07/2023,ALEXANDRE AMARAL LUCAS,01:07 AVICCENA - ALEXANDRE AMARAL SAIDA 11:10,PROB,/07/2023,AVICCENA,1,,,SAIDA,,semana,NaT,NaN,NaN,NaN
1,[03/07/2023 17:15:03] Renata Torres Bueno: 03/...,03/07/2023,RENATA TORRES BUENO,03/07 VILLA - RENATA BUENO ENTRADA 11:00 E SAI...,PROB,03/07/2023,VILLA,1,,,,,semana,NaT,NaN,NaN,NaN
2,[04/07/2023 09:25:19] Marcelo Felipe de Souza ...,04/07/2023,MARCELO FELIPE DE SOUZA BUTO,03/07 MORUMBI - BUTO ( DISPENSADO ),PROB,03/07/2023,MORUMBI,1,DISPENSADO,,,,semana,NaT,NaN,NaN,NaN
3,[04/07/2023 14:48:05] Débora Philippi Bressane...,04/07/2023,DEBORA PHILIPPI BRESSANE,04/07 ANÁLIA DÉBORA BRESSANE ENTRADA 06:00 SAI...,PROB,04/07/2023,ANALIA,1,,,,,semana,NaT,NaN,NaN,NaN
4,[04/07/2023 17:07:32] Thaís Miranda Ramos: 04/...,04/07/2023,THAIS MIRANDA RAMOS,04/07 JABAQUARA - MIRANDA SAIDA ÀS 17H,PROB,04/07/2023,JABA,1,,,SAIDA,,semana,NaT,NaN,NaN,NaN
5,[05/07/2023 06:42:15] Laysa Layna Ribeiro Lima...,05/07/2023,LAYSA LAYNA RIBEIRO LIMA,05/07 OSASCO LAYSA ENTRADA 6H,PROB,05/07/2023,OSASCO,1,,,ENTRADA,,semana,NaT,NaN,NaN,NaN
6,[05/07/2023 16:45:12] Débora Philippi Bressane...,05/07/2023,DEBORA PHILIPPI BRESSANE,05/07 JABAQUARA DÉBORA BRESSANE ENTRADA 06:00 ...,PROB,05/07/2023,JABA,1,,,,,semana,NaT,NaN,NaN,NaN
7,[06/07/2023 08:56:22] Gabriel Elias Gewehr Sal...,06/07/2023,GABRIEL ELIAS GEWEHR SALMEN RAFFOUL,05/07 JABAQUARA SALMEN DISPENSADO,PROB,05/07/2023,JABA,1,DISPENSADO,,,,semana,NaT,NaN,NaN,NaN
8,[07/07/2023 14:00:04] Fabiana Barbosa Bastos: ...,07/07/2023,FABIANA BARBOSA BASTOS,07/07 VNS - FABIANA DISPENSADO ( SAÚDE ),PROB,07/07/2023,VNS,1,DISPENSADO,,,,semana,NaT,NaN,NaN,NaN
9,[07/07/2023 21:01:54] Renata Togeiro Bastos Go...,07/07/2023,RENATA TOGEIRO BASTOS GODOY DE FARIA,07/07 MORUMBI - RENATA TOGEIRO ENTRADA 7:00 E ...,PROB,07/07/2023,MORUMBI,1,,,,,semana,NaT,NaN,NaN,NaN


In [38]:
### PROB CONTINUA DENTRO DO DF
### PROBLEMAO É SOMA DE PROBLEMAS QUE SAÍRAM DO DF: NÃO TINHAM HORA + ERRO ENTRADA OU SAÍDA

In [39]:
#DURAÇÃO DO PLANTÃO
df['DURAÇÃO'] = ''

for i in range(0, len(df)):
    diferenca_tempo = datetime.strptime(df['SAIDA'][i], '%H:%M') - datetime.strptime(df['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    df['DURAÇÃO'][i] = horas + ':' + minutos
    
df['DURAÇÃO'].unique()

array(['03:00', '02:30', '02:00', '01:30', '01:00', '00:30', '03:30',
       '04:00', '04:30', '05:00', '05:30', '00:00', '06:00', '06:30',
       '07:00'], dtype=object)

In [40]:
df['setor'] = ''

for i in range(0, len(df)):
    if df['sem_ou_fds'][i] == 'semana':
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'SEMANA - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'SEMANA - NOTURNO - SAIDA'
        else:
            df['setor'][i] = 'VERIFICAR'

    elif df['sem_ou_fds'][i] == 'fds': 
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'FDS - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'FDS - C.CIRURGICO ROTINA'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'FDS - SAIDA'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'FDS - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'  

    else:
        df['setor'][i] = 'VERIFICAR'
        
df['setor'].value_counts()

SEMANA - 6HS                    225
SEMANA - TARDE - C.CIRURGICO    173
SEMANA - NOTURNO - SAIDA        171
FDS - SAIDA                     107
FDS - 6HS                        83
FDS - C.CIRURGICO ROTINA         56
SEMANA - MANHA - C.CIRURGICO     19
FDS - PLANTAO                    16
Name: setor, dtype: int64

In [41]:
print("prob:", len(prob)) #ainda dentro de df
print("problemao:", len(problemao)) #fora de df
print("df:", len(df)) #todo df

prob: 59
problemao: 20
df: 850


In [42]:
df[df['SAIDA'] == "00:30"]

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,DURAÇÃO,setor


# Ultimos ajustes:

In [43]:
# Distancias
for i in range(0, len(df)):
    mensagem = df['mensagem'][i].upper()
    regex_D = re.findall('D[0-9]', mensagem)
    
    if len(regex_D) == 0:
        continue
    else:
        df['controle'][i] == 'PROB'

In [44]:
for i in range(0, len(df)):
    if df['DURAÇÃO'][i] == "00:00" :
        df['controle'][i] = "PROB"
        
print(df['controle'].value_counts())



        776
PROB     74
Name: controle, dtype: int64


In [45]:
df['setor'] = df['setor'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
df['unidade'] = df['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

df['unidade'] = df['unidade'].apply(lambda x: x.replace('JABA','JABAQUARA'))

df['unidade'] = df['unidade'].apply(lambda x: x.replace('SANTA','SANTA ISABEL'))

df['unidade'].unique()

#problemas com santa isabel 
#df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - SAIDA")] = "FDS - NOTURNO - SAIDA"
df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - C.CIRURGICO ROTINA")] = "FDS - MANHA - C.CIRURGICO"

In [46]:
#df['UNIDADE'] = df['UNIDADE'].apply(lambda x: x.replace('JABA','JABAQUARA'))
df['nome'] = df['nome'].apply(lambda x: x.replace('  ',' '))

#df['UNIDADE'] = df['UNIDADE'].apply(lambda x: x.replace('JABAQUARAQUARA','JABAQUARA'))
#df['UNIDADE'].unique()

In [47]:
#problemao - erro hora (mais de 1 ou nenhum), entrada saída
#prob: inclui deslocado, possível dispensa (maioria dentro de problemao) e duração 00h
#df ainda inclui prob

In [48]:
desloc = df[df['sit'] == 'DESLOCADO']
dispen = problemao[problemao['sit'] == 'DISPENSADO']
desloc.reset_index(inplace=True, drop=True)
dispen.reset_index(inplace=True, drop=True)

In [49]:
len(df)

850

In [50]:
len(desloc)

52

In [51]:
problemao = problemao[problemao['sit'] != 'DISPENSADO']
df = df[df['sit'] != 'DESLOCADO']
prob = df[df['controle'] == "PROB"]
prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
problemao.reset_index(drop=True, inplace=True)

In [52]:
len(df)

798

In [53]:
problemao

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO
0,[01/07/2023 11:15:39] Alexandre Amaral Lucas: ...,01/07/2023,ALEXANDRE AMARAL LUCAS,01:07 AVICCENA - ALEXANDRE AMARAL SAIDA 11:10,PROB,/07/2023,AVICCENA,1,,,SAIDA,,semana,NaT,NaN,NaN,NaN
1,[03/07/2023 17:15:03] Renata Torres Bueno: 03/...,03/07/2023,RENATA TORRES BUENO,03/07 VILLA - RENATA BUENO ENTRADA 11:00 E SAI...,PROB,03/07/2023,VILLA,1,,,,,semana,NaT,NaN,NaN,NaN
2,[04/07/2023 14:48:05] Débora Philippi Bressane...,04/07/2023,DEBORA PHILIPPI BRESSANE,04/07 ANÁLIA DÉBORA BRESSANE ENTRADA 06:00 SAI...,PROB,04/07/2023,ANALIA,1,,,,,semana,NaT,NaN,NaN,NaN
3,[04/07/2023 17:07:32] Thaís Miranda Ramos: 04/...,04/07/2023,THAIS MIRANDA RAMOS,04/07 JABAQUARA - MIRANDA SAIDA ÀS 17H,PROB,04/07/2023,JABA,1,,,SAIDA,,semana,NaT,NaN,NaN,NaN
4,[05/07/2023 06:42:15] Laysa Layna Ribeiro Lima...,05/07/2023,LAYSA LAYNA RIBEIRO LIMA,05/07 OSASCO LAYSA ENTRADA 6H,PROB,05/07/2023,OSASCO,1,,,ENTRADA,,semana,NaT,NaN,NaN,NaN
5,[05/07/2023 16:45:12] Débora Philippi Bressane...,05/07/2023,DEBORA PHILIPPI BRESSANE,05/07 JABAQUARA DÉBORA BRESSANE ENTRADA 06:00 ...,PROB,05/07/2023,JABA,1,,,,,semana,NaT,NaN,NaN,NaN
6,[07/07/2023 21:01:54] Renata Togeiro Bastos Go...,07/07/2023,RENATA TOGEIRO BASTOS GODOY DE FARIA,07/07 MORUMBI - RENATA TOGEIRO ENTRADA 7:00 E ...,PROB,07/07/2023,MORUMBI,1,,,,,semana,NaT,NaN,NaN,NaN
7,[08/07/2023 10:40:47] Sheila Itiroco Monte: (D...,08/07/2023,SHEILA ITIROCO MONTE,( DESLOCADO ITAIM ),PROB,/07/2023,ITAIM,1,DESLOCADO,,,,semana,NaT,NaN,NaN,NaN
8,[08/07/2023 13:07:06] Marina Borges Curralo: 0...,08/07/2023,MARINA BORGES CURRALO,08/7 MATERNIDADE MARINA SAIDA 13H,PROB,08/07/2023,MATERNIDADE,1,,,SAIDA,,fds,NaT,NaN,NaN,NaN
9,[10/07/2023 05:27:03] Maxvilano Lima de Macedo...,10/07/2023,MAXVILANO LIMA DE MACEDO,10/07 OSASCO MAXVILANO ENTRADA 04.30,PROB,10/07/2023,OSASCO,1,,,ENTRADA,,semana,NaT,NaN,NaN,NaN


In [54]:
prob ##inclúido ainda no df

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,DURAÇÃO,setor
0,[01/07/2023 12:39:03] Beatriz Antipou dos Sant...,01/07/2023,BEATRIZ ANTIPOU DOS SANTOS,01/07- JABAQUARA- ANTIPOU- SAIDA 12:38 ( EXTR...,PROB,01/07/2023,JABAQUARA,1,,extra,SAIDA,13:00,fds,1900-01-01 12:38:00,13:00,13:00,tarde,00:00,FDS - SAIDA
1,[01/07/2023 13:01:23] Cláudio Mueller Kakuda: ...,01/07/2023,CLAUDIO MUELLER KAKUDA,01/07 MATERNIDADE - KAKUDA SAIDA 12:50,PROB,01/07/2023,MATERNIDADE,1,,,SAIDA,13:00,fds,1900-01-01 12:50:00,13:00,13:00,tarde,00:00,FDS - SAIDA
2,[01/07/2023 13:04:33] Milton Olaio Neto: 01/7-...,01/07/2023,MILTON OLAIO NETO,01/7-ITAIM OLAIO SAIDA 12:50,PROB,01/07/2023,ITAIM,1,,,SAIDA,13:00,fds,1900-01-01 12:50:00,13:00,13:00,tarde,00:00,FDS - SAIDA
3,[01/07/2023 18:35:18] Daniel Masuzaki Wong: 0...,01/07/2023,DANIEL MASUZAKI WONG,01/07 MORUMBI - WONG SAIDA 18:35,PROB,01/07/2023,MORUMBI,1,,,SAIDA,19:00,fds,1900-01-01 18:35:00,19:00,19:00,noturno saida,00:00,FDS - PLANTAO
4,[01/07/2023 18:53:51] Leandro Oliveira Martins...,01/07/2023,LEANDRO OLIVEIRA MARTINS CORTES,01/07 ANALIA - CÔRTES SAIDA 18:50,PROB,01/07/2023,ANALIA,1,,,SAIDA,19:00,fds,1900-01-01 18:50:00,19:00,19:00,noturno saida,00:00,FDS - PLANTAO
5,[01/07/2023 18:54:54] Rodrigo Martins Pereira:...,01/07/2023,RODRIGO MARTINS PEREIRA,01/07 VILLA - PEREIRA SAIDA 18:48,PROB,01/07/2023,VILLA,1,,,SAIDA,19:00,fds,1900-01-01 18:48:00,19:00,19:00,noturno saida,00:00,FDS - PLANTAO
6,[01/07/2023 18:58:46] Victor Sapountzakis Ribe...,01/07/2023,VICTOR SAPOUNTZAKIS RIBEIRO,01/07 JABAQUARA - SAPOUNTZAKIS SAIDA 18:58,PROB,01/07/2023,JABAQUARA,1,,,SAIDA,19:00,fds,1900-01-01 18:58:00,19:00,19:00,noturno saida,00:00,FDS - PLANTAO
7,[02/07/2023 12:41:19] Thales Luvisotto Neder: ...,02/07/2023,THALES LUVISOTTO NEDER,02/07 MORUMBI - THALES SAIDA 12:40 ( D3B - EX...,PROB,02/07/2023,MORUMBI,1,,extra,SAIDA,13:00,fds,1900-01-01 12:40:00,13:00,13:00,tarde,00:00,FDS - SAIDA
8,[02/07/2023 12:42:34] Maxvilano Lima de Macedo...,02/07/2023,MAXVILANO LIMA DE MACEDO,02/07 MORUMBI - MAX SAIDA 12:40 D3R,PROB,02/07/2023,MORUMBI,1,,,SAIDA,13:00,fds,1900-01-01 12:40:00,13:00,13:00,tarde,00:00,FDS - SAIDA
9,[03/07/2023 12:52:50] Daniel Masuzaki Wong: 0...,03/07/2023,DANIEL MASUZAKI WONG,03/07 JABAQUARA - WONG SAIDA 12:45 ( PÓS PLAN...,PROB,03/07/2023,JABAQUARA,1,,,SAIDA,13:00,semana,1900-01-01 12:45:00,13:00,13:00,tarde,00:00,SEMANA - TARDE - C.CIRURGICO


In [55]:
desloc

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,DURAÇÃO,setor
0,[01/07/2023 05:30:43] Gabriela Santos Menezes:...,01/07/2023,GABRIELA SANTOS MENEZES,01/07 ANÁLIA - G.MENEZES ENTRADA 05:30 ( DESL...,PROB,01/07/2023,ANALIA,2,DESLOCADO,,ENTRADA,05:30,fds,1900-01-01 05:30:00,05:30,07:00,6h,01:30,FDS - 6HS
1,[01/07/2023 06:55:12] Denio Lopes de Almeida: ...,01/07/2023,DENIO LOPES DE ALMEIDA,01/07 OSASCO - DÊNIO ENTRADA 06:00 ( DESLOCAD...,PROB,01/07/2023,OSASCO,2,DESLOCADO,,ENTRADA,06:00,fds,1900-01-01 06:00:00,06:00,07:00,6h,01:00,FDS - 6HS
2,[01/07/2023 10:27:51] Caroline Firmino Lemes d...,01/07/2023,CAROLINE FIRMINO LEMES DA SILVA,01/07 OSASCO - LEMES SAIDA 10:25 ( DESLOCADO ...,PROB,01/07/2023,OSASCO,2,DESLOCADO,,SAIDA,10:30,fds,1900-01-01 10:25:00,07:00,10:30,manha,03:30,FDS - C.CIRURGICO ROTINA
3,[01/07/2023 13:13:59] Denio Lopes de Almeida: ...,01/07/2023,DENIO LOPES DE ALMEIDA,01/07 OSASCO - DÊNIO SAIDA 13:10 ( DESLOCADO ...,PROB,01/07/2023,OSASCO,2,DESLOCADO,,SAIDA,13:30,fds,1900-01-01 13:10:00,13:00,13:30,tarde,00:30,FDS - SAIDA
4,[01/07/2023 13:23:26] Ronald Figueiredo Gomes:...,01/07/2023,RONALD FIGUEIREDO GOMES,01/07 JABAQUARA - RONALD SAIDA 13:15 ( PÓS PL...,PROB,01/07/2023,JABAQUARA,2,DESLOCADO,,SAIDA,13:30,fds,1900-01-01 13:15:00,13:00,13:30,tarde,00:30,FDS - SAIDA
5,[01/07/2023 15:00:58] Gabriela Santos Menezes:...,01/07/2023,GABRIELA SANTOS MENEZES,01/07 ANÁLIA - G.MENEZES SAIDA 15:00 ( DESLOC...,PROB,01/07/2023,ANALIA,2,DESLOCADO,,SAIDA,15:00,fds,1900-01-01 15:00:00,13:00,15:00,tarde,02:00,FDS - SAIDA
6,[01/07/2023 17:40:55] André da Costa Aguiar Ve...,01/07/2023,ANDRE DA COSTA AGUIAR VENTURA,01/07 ANALIA VENTURA SAIDA 17:40 ( DESLOCAD...,PROB,01/07/2023,ANALIA,2,DESLOCADO,,SAIDA,18:00,fds,1900-01-01 17:40:00,13:00,18:00,tarde,05:00,FDS - SAIDA
7,[01/07/2023 20:58:55] Bianca Priscila Buniotti...,01/07/2023,BIANCA PRISCILA BUNIOTTI,01/07 VILLA - BIANCA SAIDA 16:45 ( DESLOCADO ...,PROB,01/07/2023,VILLA,2,DESLOCADO,,SAIDA,17:00,fds,1900-01-01 16:45:00,13:00,17:00,tarde,04:00,FDS - SAIDA
8,[03/07/2023 04:35:22] Kaiê Fernandes: 03/07 Os...,03/07/2023,KAIE FERNANDES,03/07 OSASCO KAIE ENTRADA 4:20 ( DESLOCADO IT...,PROB,03/07/2023,OSASCO,2,DESLOCADO,,ENTRADA,04:00,semana,1900-01-01 04:20:00,04:00,07:00,6h,03:00,SEMANA - 6HS
9,[03/07/2023 05:36:37] Rodrigo Antonio Rey Gonz...,03/07/2023,RODRIGO ANTONIO REY GONZALEZ,03/07 OSASCO - REY ENTRADA 05:35 ( DESLOCADO ...,PROB,03/07/2023,OSASCO,2,DESLOCADO,,ENTRADA,05:30,semana,1900-01-01 05:35:00,05:30,07:00,6h,01:30,SEMANA - 6HS


In [56]:
dispen

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO
0,[04/07/2023 09:25:19] Marcelo Felipe de Souza ...,04/07/2023,MARCELO FELIPE DE SOUZA BUTO,03/07 MORUMBI - BUTO ( DISPENSADO ),PROB,03/07/2023,MORUMBI,1,DISPENSADO,,,,semana,NaT,NaN,NaN,NaN
1,[06/07/2023 08:56:22] Gabriel Elias Gewehr Sal...,06/07/2023,GABRIEL ELIAS GEWEHR SALMEN RAFFOUL,05/07 JABAQUARA SALMEN DISPENSADO,PROB,05/07/2023,JABA,1,DISPENSADO,,,,semana,NaT,NaN,NaN,NaN
2,[07/07/2023 14:00:04] Fabiana Barbosa Bastos: ...,07/07/2023,FABIANA BARBOSA BASTOS,07/07 VNS - FABIANA DISPENSADO ( SAÚDE ),PROB,07/07/2023,VNS,1,DISPENSADO,,,,semana,NaT,NaN,NaN,NaN
3,[08/07/2023 07:31:30] Isabela Mattarelli Souza...,08/07/2023,ISABELA MATTARELLI SOUZA CARLI,08/07 - ITAIM - MATTARELLI DISPENSADO PÓS PLANTÃO,PROB,08/07/2023,ITAIM,1,DISPENSADO,,,,fds,NaT,NaN,NaN,NaN
4,[08/07/2023 07:32:36] Nathalia Fanucci Oliveir...,08/07/2023,NATHALIA FANUCCI OLIVEIRA DI JORGE,08/07 OSASCO NATHÁLIA DISPENSADO ( SAÚDE ),PROB,08/07/2023,OSASCO,1,DISPENSADO,,,,fds,NaT,NaN,NaN,NaN
5,[08/07/2023 08:52:00] Alexsander Takashi Hamad...,08/07/2023,ALEXSANDER TAKASHI HAMADA,08/07 - VILLA - ALEX DISPENSADO NO DIA ANTER...,PROB,08/07/2023,VILLA,2,DISPENSADO,,,,fds,NaT,NaN,NaN,NaN
6,[08/07/2023 09:28:33] Gabriel Elias Gewehr Sal...,08/07/2023,GABRIEL ELIAS GEWEHR SALMEN RAFFOUL,08/07 JABAQUARA SALMEN DISPENSADOSAÚDE,PROB,08/07/2023,JABA,1,DISPENSADO,,,,fds,NaT,NaN,NaN,NaN


In [57]:
df = df[df['controle'] != "PROB"]
df.reset_index(drop=True, inplace=True)

In [58]:
len(df)

776

In [59]:
len(prob)

22

In [60]:
problemas_final = pd.concat([problemao, prob])
problemas_final.reset_index(drop=True, inplace=True)

In [61]:
len(problemas_final)

35

In [62]:
problemas_final

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,DURAÇÃO,setor
0,[01/07/2023 11:15:39] Alexandre Amaral Lucas: ...,01/07/2023,ALEXANDRE AMARAL LUCAS,01:07 AVICCENA - ALEXANDRE AMARAL SAIDA 11:10,PROB,/07/2023,AVICCENA,1,,,SAIDA,,semana,NaT,NaN,NaN,NaN,NaN,NaN
1,[03/07/2023 17:15:03] Renata Torres Bueno: 03/...,03/07/2023,RENATA TORRES BUENO,03/07 VILLA - RENATA BUENO ENTRADA 11:00 E SAI...,PROB,03/07/2023,VILLA,1,,,,,semana,NaT,NaN,NaN,NaN,NaN,NaN
2,[04/07/2023 14:48:05] Débora Philippi Bressane...,04/07/2023,DEBORA PHILIPPI BRESSANE,04/07 ANÁLIA DÉBORA BRESSANE ENTRADA 06:00 SAI...,PROB,04/07/2023,ANALIA,1,,,,,semana,NaT,NaN,NaN,NaN,NaN,NaN
3,[04/07/2023 17:07:32] Thaís Miranda Ramos: 04/...,04/07/2023,THAIS MIRANDA RAMOS,04/07 JABAQUARA - MIRANDA SAIDA ÀS 17H,PROB,04/07/2023,JABA,1,,,SAIDA,,semana,NaT,NaN,NaN,NaN,NaN,NaN
4,[05/07/2023 06:42:15] Laysa Layna Ribeiro Lima...,05/07/2023,LAYSA LAYNA RIBEIRO LIMA,05/07 OSASCO LAYSA ENTRADA 6H,PROB,05/07/2023,OSASCO,1,,,ENTRADA,,semana,NaT,NaN,NaN,NaN,NaN,NaN
5,[05/07/2023 16:45:12] Débora Philippi Bressane...,05/07/2023,DEBORA PHILIPPI BRESSANE,05/07 JABAQUARA DÉBORA BRESSANE ENTRADA 06:00 ...,PROB,05/07/2023,JABA,1,,,,,semana,NaT,NaN,NaN,NaN,NaN,NaN
6,[07/07/2023 21:01:54] Renata Togeiro Bastos Go...,07/07/2023,RENATA TOGEIRO BASTOS GODOY DE FARIA,07/07 MORUMBI - RENATA TOGEIRO ENTRADA 7:00 E ...,PROB,07/07/2023,MORUMBI,1,,,,,semana,NaT,NaN,NaN,NaN,NaN,NaN
7,[08/07/2023 10:40:47] Sheila Itiroco Monte: (D...,08/07/2023,SHEILA ITIROCO MONTE,( DESLOCADO ITAIM ),PROB,/07/2023,ITAIM,1,DESLOCADO,,,,semana,NaT,NaN,NaN,NaN,NaN,NaN
8,[08/07/2023 13:07:06] Marina Borges Curralo: 0...,08/07/2023,MARINA BORGES CURRALO,08/7 MATERNIDADE MARINA SAIDA 13H,PROB,08/07/2023,MATERNIDADE,1,,,SAIDA,,fds,NaT,NaN,NaN,NaN,NaN,NaN
9,[10/07/2023 05:27:03] Maxvilano Lima de Macedo...,10/07/2023,MAXVILANO LIMA DE MACEDO,10/07 OSASCO MAXVILANO ENTRADA 04.30,PROB,10/07/2023,OSASCO,1,,,ENTRADA,,semana,NaT,NaN,NaN,NaN,NaN,NaN


### PESQUISANDO PERÍODOS A EXCLUIR PÓS 13H

In [63]:
df['dt'] = pd.to_datetime(df['SAIDA'], format="%H:%M")

In [64]:
df['conferir'] = ""

In [65]:
for i in range(0, len(df)):
    if df['dt'][i].hour < 13 and df['dt'][i].hour !=0 and df['tipo'][i] == "SAIDA":
        df['conferir'][i] = "SIM"
    elif df['dt'][i].hour == 13 and df['dt'][i].minute == 0 and  df['tipo'][i] == "SAIDA":
        df['conferir'][i] = "SIM"
    else:
        pass

In [66]:
conferir = df[df['conferir'] == 'SIM']

In [67]:
conferir.reset_index(inplace=True, drop=True)
conferir

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,DURAÇÃO,setor,dt,conferir
0,[01/07/2023 09:49:24] Felipe Coelho de Souza: ...,01/07/2023,FELIPE COELHO DE SOUZA,01/07 ITAIM - COELHO SAIDA 9:48,,01/07/2023,ITAIM,1,,,SAIDA,10:00,fds,1900-01-01 09:48:00,07:00,10:00,manha,03:00,FDS - C.CIRURGICO ROTINA,1900-01-01 10:00:00,SIM
1,[01/07/2023 10:12:00] Karen Peixoto Ricci: 01/...,01/07/2023,KAREN PEIXOTO RICCI,01/07 ITAIM KAREN SAIDA 10:00,,01/07/2023,ITAIM,1,,,SAIDA,10:00,fds,1900-01-01 10:00:00,07:00,10:00,manha,03:00,FDS - C.CIRURGICO ROTINA,1900-01-01 10:00:00,SIM
2,[01/07/2023 10:23:11] Rodrigo Achui Haga: 01/0...,01/07/2023,RODRIGO ACHUI HAGA,01/07 OSASCO - HAGA SAIDA 10:20,,01/07/2023,OSASCO,1,,,SAIDA,10:30,fds,1900-01-01 10:20:00,07:00,10:30,manha,03:30,FDS - C.CIRURGICO ROTINA,1900-01-01 10:30:00,SIM
3,[01/07/2023 10:40:02] Rodrigo Miyashiro Nunes ...,01/07/2023,RODRIGO MIYASHIRO NUNES DOS SANTOS,01/07 ITAIM - MIYASHIRO SAIDA 10:40,,01/07/2023,ITAIM,1,,,SAIDA,11:00,fds,1900-01-01 10:40:00,07:00,11:00,manha,04:00,FDS - C.CIRURGICO ROTINA,1900-01-01 11:00:00,SIM
4,[01/07/2023 11:01:34] Fabiana Barbosa Bastos: ...,01/07/2023,FABIANA BARBOSA BASTOS,01/07 ITAIM - FABIANA SAIDA 11:00,,01/07/2023,ITAIM,1,,,SAIDA,11:00,fds,1900-01-01 11:00:00,07:00,11:00,manha,04:00,FDS - C.CIRURGICO ROTINA,1900-01-01 11:00:00,SIM
5,[01/07/2023 11:20:37] Ricardo Barbosa Viana Fi...,01/07/2023,RICARDO BARBOSA VIANA FILHO,01/07 SANTA ISABEL - BARBOSA SAIDA 11:15 ( EX...,,01/07/2023,SANTA ISABEL,1,,extra,SAIDA,11:30,fds,1900-01-01 11:15:00,07:00,11:30,manha,04:30,FDS - MANHA - C.CIRURGICO,1900-01-01 11:30:00,SIM
6,[01/07/2023 11:21:08] Francisco José Lucena Be...,01/07/2023,FRANCISCO JOSE LUCENA BEZERRA,01/07 SANTA ISABEL- LUCENA- SAIDA 11:15,,01/07/2023,SANTA ISABEL,1,,,SAIDA,11:30,fds,1900-01-01 11:15:00,07:00,11:30,manha,04:30,FDS - MANHA - C.CIRURGICO,1900-01-01 11:30:00,SIM
7,[01/07/2023 11:40:45] Anna Beatriz Lima Do Val...,01/07/2023,ANNA BEATRIZ LIMA DO VALLE ASTUR,01/07 SANTA ISABEL - ASTUR SAIDA 11:40 ( EXTR...,,01/07/2023,SANTA ISABEL,1,,extra,SAIDA,12:00,fds,1900-01-01 11:40:00,07:00,12:00,manha,05:00,FDS - MANHA - C.CIRURGICO,1900-01-01 12:00:00,SIM
8,[01/07/2023 12:19:34] Ricardo Barbosa Viana Fi...,01/07/2023,RICARDO BARBOSA VIANA FILHO,01/07 SANTA ISABEL - BARBOSA SAIDA 12:15,,01/07/2023,SANTA ISABEL,1,,,SAIDA,12:30,fds,1900-01-01 12:15:00,07:00,12:30,manha,05:30,FDS - MANHA - C.CIRURGICO,1900-01-01 12:30:00,SIM
9,[02/07/2023 11:52:41] Beatriz Antipou dos Sant...,02/07/2023,BEATRIZ ANTIPOU DOS SANTOS,02/07- ITAIM- ANTIPOU- SAIDA 11:50 - D3B,,02/07/2023,ITAIM,1,,,SAIDA,12:00,fds,1900-01-01 11:50:00,07:00,12:00,manha,05:00,FDS - C.CIRURGICO ROTINA,1900-01-01 12:00:00,SIM


In [68]:
escala = pd.read_excel('Junho.xlsx')
escala.columns = ['nome','data','inicio','duracao']
escala = escala.replace("-",None, regex=True)
escala = escala.dropna()
escala['data'] = escala['data'].dt.strftime('%d/%m')
escala['nome'] = escala['nome'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
escala.reset_index(inplace=True, drop=True)


In [69]:
atencao = pd.DataFrame(columns=['nome','data'])

In [70]:
for i in range(0,len(escala)):
    for j in range(0,len(conferir)):
        if escala['nome'][i] == conferir['nome'][j] and escala['data'][i]+"/2023" == conferir['data'][j] and\
        escala['inicio'][i].hour == 13:
            atencao.loc[j] = [conferir['nome'][j], conferir['data'][j]]
        

In [71]:
atencao.reset_index(inplace=True, drop=True)
atencao

,nome,data


# Salvando:

In [72]:
df.to_excel("df_ok1.xlsx", index=False) #tudo que está certo
problemas_final.to_excel('df_giovarever1.xlsx', index=False) #tudo que é problema (inclui principalmente duração 00h do prob + oq sobrou problemao)

In [73]:
desloc.to_excel("desloc.xlsx", index=False) #todos deslocados
dispen.to_excel("dispen.xlsx", index=False) #todos dispensados

In [74]:
atencao.to_excel("apagar13.xlsx", index=False) #todos para apagar a tarde

In [75]:
conferir

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,DURAÇÃO,setor,dt,conferir
0,[01/07/2023 09:49:24] Felipe Coelho de Souza: ...,01/07/2023,FELIPE COELHO DE SOUZA,01/07 ITAIM - COELHO SAIDA 9:48,,01/07/2023,ITAIM,1,,,SAIDA,10:00,fds,1900-01-01 09:48:00,07:00,10:00,manha,03:00,FDS - C.CIRURGICO ROTINA,1900-01-01 10:00:00,SIM
1,[01/07/2023 10:12:00] Karen Peixoto Ricci: 01/...,01/07/2023,KAREN PEIXOTO RICCI,01/07 ITAIM KAREN SAIDA 10:00,,01/07/2023,ITAIM,1,,,SAIDA,10:00,fds,1900-01-01 10:00:00,07:00,10:00,manha,03:00,FDS - C.CIRURGICO ROTINA,1900-01-01 10:00:00,SIM
2,[01/07/2023 10:23:11] Rodrigo Achui Haga: 01/0...,01/07/2023,RODRIGO ACHUI HAGA,01/07 OSASCO - HAGA SAIDA 10:20,,01/07/2023,OSASCO,1,,,SAIDA,10:30,fds,1900-01-01 10:20:00,07:00,10:30,manha,03:30,FDS - C.CIRURGICO ROTINA,1900-01-01 10:30:00,SIM
3,[01/07/2023 10:40:02] Rodrigo Miyashiro Nunes ...,01/07/2023,RODRIGO MIYASHIRO NUNES DOS SANTOS,01/07 ITAIM - MIYASHIRO SAIDA 10:40,,01/07/2023,ITAIM,1,,,SAIDA,11:00,fds,1900-01-01 10:40:00,07:00,11:00,manha,04:00,FDS - C.CIRURGICO ROTINA,1900-01-01 11:00:00,SIM
4,[01/07/2023 11:01:34] Fabiana Barbosa Bastos: ...,01/07/2023,FABIANA BARBOSA BASTOS,01/07 ITAIM - FABIANA SAIDA 11:00,,01/07/2023,ITAIM,1,,,SAIDA,11:00,fds,1900-01-01 11:00:00,07:00,11:00,manha,04:00,FDS - C.CIRURGICO ROTINA,1900-01-01 11:00:00,SIM
5,[01/07/2023 11:20:37] Ricardo Barbosa Viana Fi...,01/07/2023,RICARDO BARBOSA VIANA FILHO,01/07 SANTA ISABEL - BARBOSA SAIDA 11:15 ( EX...,,01/07/2023,SANTA ISABEL,1,,extra,SAIDA,11:30,fds,1900-01-01 11:15:00,07:00,11:30,manha,04:30,FDS - MANHA - C.CIRURGICO,1900-01-01 11:30:00,SIM
6,[01/07/2023 11:21:08] Francisco José Lucena Be...,01/07/2023,FRANCISCO JOSE LUCENA BEZERRA,01/07 SANTA ISABEL- LUCENA- SAIDA 11:15,,01/07/2023,SANTA ISABEL,1,,,SAIDA,11:30,fds,1900-01-01 11:15:00,07:00,11:30,manha,04:30,FDS - MANHA - C.CIRURGICO,1900-01-01 11:30:00,SIM
7,[01/07/2023 11:40:45] Anna Beatriz Lima Do Val...,01/07/2023,ANNA BEATRIZ LIMA DO VALLE ASTUR,01/07 SANTA ISABEL - ASTUR SAIDA 11:40 ( EXTR...,,01/07/2023,SANTA ISABEL,1,,extra,SAIDA,12:00,fds,1900-01-01 11:40:00,07:00,12:00,manha,05:00,FDS - MANHA - C.CIRURGICO,1900-01-01 12:00:00,SIM
8,[01/07/2023 12:19:34] Ricardo Barbosa Viana Fi...,01/07/2023,RICARDO BARBOSA VIANA FILHO,01/07 SANTA ISABEL - BARBOSA SAIDA 12:15,,01/07/2023,SANTA ISABEL,1,,,SAIDA,12:30,fds,1900-01-01 12:15:00,07:00,12:30,manha,05:30,FDS - MANHA - C.CIRURGICO,1900-01-01 12:30:00,SIM
9,[02/07/2023 11:52:41] Beatriz Antipou dos Sant...,02/07/2023,BEATRIZ ANTIPOU DOS SANTOS,02/07- ITAIM- ANTIPOU- SAIDA 11:50 - D3B,,02/07/2023,ITAIM,1,,,SAIDA,12:00,fds,1900-01-01 11:50:00,07:00,12:00,manha,05:00,FDS - C.CIRURGICO ROTINA,1900-01-01 12:00:00,SIM
